# Ground Truth Generation (Initial Captures)

This notebook generates ground truth maps using a simple watershed algorithm and user input, it will generate 

In [1]:
from spectral import *
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import pickle
from os.path import exists
from matplotlib import cm

In [6]:
# make sure to use radiance, for some reason reflectance doesn't work properly
fn = "processed-data/sardi-capture-final/final/radiance/2023_03_28/food_and_waste/2023_03_28-02_26_42.nc"
gnd_t_fn = fn.replace('radiance','reflectance').split('.')[0] + "-gnd_t.pkl"

In [7]:
def process_into_rgb(loc_ds):
    """
    Processes a given dataset into an rgb image for display
    """
    loc_dc = np.array(loc_ds['datacube'])
    loc_binned_wavelengths = np.array(loc_ds['wavelength'])
    loc_image = np.zeros((loc_dc.shape[1],loc_dc.shape[2],3), np.uint8)

    loc_red_wavelength = 620
    loc_green_wavelength = 560
    loc_blue_wavelength = 470

    loc_red_layer = loc_dc[np.argmin(np.abs(loc_binned_wavelengths-loc_red_wavelength)),:,:]  
    loc_green_layer = loc_dc[np.argmin(np.abs(loc_binned_wavelengths-loc_green_wavelength)),:,:] 
    loc_blue_layer = loc_dc[np.argmin(np.abs(loc_binned_wavelengths-loc_blue_wavelength)),:,:] 

    # loc_red_layer = loc_dc[120,:,:]  
    # loc_green_layer = loc_dc[80,:,:] 
    # loc_blue_layer = loc_dc[20,:,:] 

    loc_image[:,:,0] = loc_red_layer
    loc_image[:,:,1] = loc_green_layer
    loc_image[:,:,2] = loc_blue_layer

    # scale the values to within the central 2 to 98 percent values to remove outlier readings
    loc_vmax = np.nanpercentile(loc_image, 98)
    loc_vmin = np.nanpercentile(loc_image, 2)
    loc_image = ((loc_image.astype("f8") - loc_vmin) / (loc_vmax - loc_vmin)).astype("f4")
    loc_image = np.minimum(np.maximum(loc_image, 0), 1)

    loc_image *= 255
    loc_image = loc_image.astype(np.uint8)
    
    return loc_image

In [8]:
ds = nc.Dataset(fn)

In [9]:
image = process_into_rgb(ds)

In [10]:
# force recreation of ground truth files
force = True

In [12]:
def create_rgb(i):
    return tuple(np.array(cm.tab10(i)[:3])*255)

def on_click(event,x,y,flags,params):
    global marks_updated

    if event == cv.EVENT_LBUTTONDOWN:                         
        # add mark to image that gets passed to the watershed algo
        cv.circle(marker_img,(x,y),2,(current_marker),-1) 

        # add mark to image for user to see
        cv.circle(img_copy,(x,y),2,colors[current_marker],-1)

        marks_updated = True 

if not exists(gnd_t_fn) or force:
    img_copy=np.copy(image)
    marker_img=np.zeros(image.shape[:2],dtype=np.int32)
    segments = np.zeros(image.shape,dtype=np.uint8)
    ground_truth = np.zeros(image.shape[:2],dtype=np.uint8)

    cm.tab10(0)

    colors = []
    for i in range(10):
        colors.append(create_rgb(i))
    colors[1]=(0,0,0)
    
    n_marker = 9
    current_marker = 1
    marks_updated = False

    # create display window
    cv.namedWindow('Image')
    cv.setMouseCallback('Image', on_click)

    while True:
        
        cv.imshow('Watershed segments',segments)
        cv.imshow('Image',img_copy)

        k = cv.waitKey(1)

        # jump out of loop and close all windows with ESC
        if k == 27:
            break

        # clear the images with c
        elif k == ord('c'):
            img_copy = image.copy()
            marker_img = np.zeros(image.shape[:2],dtype = np.int32)
            segments = np.zeros(image.shape,dtype = np.uint8) 

        # update the colour choice, keys [1-9] 
        elif k > 0 and chr(k).isdigit():
            current_marker = int(chr(k))

        # update the watershed fill based on the marks
        if marks_updated:
            marker_img_copy = marker_img.copy()
            cv.watershed(image,marker_img_copy)
            segments = np.zeros(image.shape,dtype = np.uint8)
            
            for color_ind in range(n_marker):
                segments[marker_img_copy == (color_ind)] = colors[color_ind]
                ground_truth[marker_img_copy == (color_ind)] = color_ind

    # close all windows
    cv.destroyAllWindows()

    # save ground truth for future use
    with open(gnd_t_fn, 'wb') as out_file:
        pickle.dump(ground_truth, out_file)